In [67]:
import pandas as pd

In [68]:
def read_dvsa0201(sheets_name):
    df_raw = pd.read_excel(r"C:\Users\cheny\OneDrive - Durham University\其他\UK Driving\dvsa0201.xlsx", sheet_name=sheets_name, header=None)
    df_raw.drop(df_raw.head(8).index,inplace=True)
    df_raw.columns=["survey_date","male_take","male_pass","male_r","空1","female_take","female_pass","female_r","空2","take","pass","r"]
    df=  df_raw.drop(['空1', '空2'], axis=1).dropna(subset= ["survey_date"])
    # df['survey_date'].astype(str)
    df["a"] = df['survey_date'].str.slice(0,1)
    df = df.dropna(subset= ["a","r"]).reset_index()
    
    # df21.shape[0]
    df= df.rename(columns={'survey_date':'center_name'})

    df_merge = pd.merge(df,zip,on="center_name", how = "left")
    return df_merge



In [69]:
zip = pd.read_csv("center_zip.csv")
zip.columns=["center_name","zip"]

merge_file = []
for i in ["2021-22", "2020-21", "2019-20", "2018-19","2017-18",]:
    table = read_dvsa0201(i)
    merge_file.append(table) 
    
result = pd.concat(merge_file)


In [70]:
result

,index,center_name,male_take,male_pass,male_r,female_take,female_pass,female_r,take,pass,r,a,zip
0,20,Aberdeen North,1887,1095,58.028617,2082,1117,53.650336,3976,2213,55.658954,A,NaN
1,34,Aberdeen South (Cove),1703,1054,61.890781,1674,995,59.438471,3385,2052,60.620384,A,AB12 3GQ
2,48,Aberfeldy,54,43,79.62963,48,30,62.5,102,73,71.568627,A,PH15 2BJ
3,63,Abergavenny,2579,1615,62.621171,2647,1542,58.254628,5226,3157,60.409491,A,NP7 5HT
4,78,Aberystwyth (Park Avenue),616,410,66.558442,647,386,59.659969,1263,796,63.024545,A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,5131,Yeading (London),..,..,28.571429,..,..,22.222222,..,..,25,Y,NaN
350,5146,Yeovil,2510,1624,64.701195,2796,1626,58.154506,5306,3250,61.251413,Y,NaN
351,5161,York,3048,1660,54.461942,3906,1863,47.695853,6954,3523,50.66149,Y,NaN
352,5170,Z Failsworth (Closed),1739,689,39.620472,2271,718,31.616028,4010,1407,35.087282,Z,NaN


In [74]:
result.to_csv("result.csv")

## manual


In [75]:
result['take'] = pd.to_numeric(result['take'], errors='coerce')
result['male_take'] = pd.to_numeric(result['male_take'], errors='coerce')
result['female_take'] = pd.to_numeric(result['female_take'], errors='coerce')
result['female_pass'] = pd.to_numeric(result['female_pass'], errors='coerce')
result['male_pass'] = pd.to_numeric(result['male_pass'], errors='coerce')
result['pass'] = pd.to_numeric(result['pass'], errors='coerce')

In [76]:
result.dtypes

index            int64
center_name     object
male_take      float64
male_pass      float64
male_r          object
female_take    float64
female_pass    float64
female_r        object
take           float64
pass           float64
r               object
a               object
zip             object
dtype: object

In [77]:
nuts = pd.read_csv("nuts_zip.csv")
nuts["zip"] = nuts["zip"].str.replace("  "," ")

In [ ]:
df_data = pd.merge(result,nuts,on="zip", how = "left")

In [ ]:
df_data =df_data.groupby('nuts').sum()
df_data['nuts218cd']= df_data.index

In [ ]:
df_data

In [ ]:
df_data['male_r']= df_data['male_pass']/df_data['male_take']
df_data['female_r']= df_data['female_pass']/df_data['female_take']
df_data['r']= df_data['pass']/df_data['take']

In [ ]:
import pandas as pd
import json
from urllib.request import urlopen
import numpy as np

#Load GeoJson 
with urlopen('https://opendata.arcgis.com/datasets/48b6b85bb7ea43699ee85f4ecd12fd36_4.geojson') as response:
    counties = json.load(response)
 
# Load data to be charted    
dummy_data=pd.read_csv('https://opendata.arcgis.com/datasets/48b6b85bb7ea43699ee85f4ecd12fd36_0.csv?outSR=%7B%22latestWkid%22%3A27700%2C%22wkid%22%3A27700%7D')
#add dummy data
dummy_data = pd.merge(dummy_data,df_data,on="nuts218cd", how = "left")
#With Plotly


In [ ]:
dummy_data

In [ ]:
import plotly.express as px
import plotly.io as pio
from geojson_rewind import rewind

#Make the rings clockwwise (to make it compatible with plotly)    
counties_corrected=rewind(counties,rfc7946=False)

In [57]:
fig = px.choropleth(dummy_data, geojson=counties_corrected, locations='nuts218cd', featureidkey="properties.nuts218cd", color='take',
                            color_continuous_scale="puor", labels={'label name':'label name'},
                            scope="europe") #blackbody geyser  earth

fig.update_geos(fitbounds="locations", visible=False).write_image('fig4.png', scale=1, width=1000, height=800)


In [58]:
fig = px.choropleth(dummy_data, geojson=counties_corrected, locations='nuts218cd', featureidkey="properties.nuts218cd", color='r',
                            color_continuous_scale="earth", labels={'label name':'label name'},
                            scope="europe") #blackbody geyser  earth

fig.update_geos(fitbounds="locations", visible=False).write_image('fig1.png', scale=1, width=1000, height=800)

In [59]:
fig = px.choropleth(dummy_data, geojson=counties_corrected, locations='nuts218cd', featureidkey="properties.nuts218cd", color='male_r',
                            color_continuous_scale="earth", labels={'label name':'label name'}, 
                            scope="europe") #blackbody geyser 

fig.update_geos(fitbounds="locations", visible=False).write_image('fig2.png', scale=1, width=1000, height=800)

In [60]:
fig = px.choropleth(dummy_data, geojson=counties_corrected, locations='nuts218cd', featureidkey="properties.nuts218cd", color='female_r',
                            color_continuous_scale="earth", labels={'label name':'label name'}, 
                            scope="europe") #blackbody geyser 

fig.update_geos(fitbounds="locations", visible=False).write_image('fig3.png', scale=1, width=1000, height=800)

In [ ]:
dummy_data.to_csv("dummy.csv")